# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234879884459                   -0.50    7.0
  2   -7.250341216147       -1.81       -1.40    1.0
  3   -7.251290633192       -3.02       -2.11    4.0
  4   -7.251196067489   +   -4.02       -2.08    4.0
  5   -7.251330467184       -3.87       -2.70    2.0
  6   -7.251337999202       -5.12       -3.12    2.0
  7   -7.251338775662       -6.11       -4.03    1.0
  8   -7.251338792264       -7.78       -4.15    4.0
  9   -7.251338798241       -8.22       -4.64    3.0
 10   -7.251338798492       -9.60       -4.88    1.0
 11   -7.251338798698       -9.69       -5.59    3.0
 12   -7.251338798704      -11.21       -5.99    2.0
 13   -7.251338798704      -12.24       -6.41    3.0
 14   -7.251338798705      -12.65       -7.12    2.0
 15   -7.251338798705      -14.15       -7.47    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06317526688648221
[ Info: Arnoldi iteration step 2: normres = 0.3250517017949442
[ Info: Arnoldi iteration step 3: normres = 0.2801637983059518
[ Info: Arnoldi iteration step 4: normres = 0.9858706911266695
[ Info: Arnoldi iteration step 5: normres = 0.5366170181649372
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.78e-01, 5.34e-02, 2.60e-01, 2.72e-01, 2.41e-02)
[ Info: Arnoldi iteration step 6: normres = 0.41519623654586096
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.59e-01, 2.18e-01, 2.92e-01, 8.42e-02, 7.89e-02)
[ Info: Arnoldi iteration step 7: normres = 0.10003074503488575
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.96e-03, 2.68e-02, 9.40e-03, 3.11e-02, 7.53e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10452545992878604
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (3.65e-